# Tutorial 2: Arithmetic Secret Sharing
Arithmetic secret sharing is used in secure two-party computation, where each participant holds the shared value of the data. In this way the data does not leak information during the calculation process. At present, our model and functions are designed based on semi-honest parties.
To use arithmetic secret sharing for secure two-party computation, we import the following packages

In [1]:
# import the libraries
from model.mpc.semi_honest_party import SemiHonestCS
from crypto.primitives.arithmetic_secret_sharing.arithmetic_shared_ring_tensor import ArithmeticSharedRingTensor
from common.tensor.ring_tensor import RingTensor
from crypto.primitives.beaver.matrix_triples import MatrixTriples

import torch

```SemiHonestCS``` is the two semi-honest party. ```arithmetic_shared_ring_tensor``` is the main package that we use. ```RingTensor``` is the main data structure that we use. ```BeaverProvider``` is the triple provider we use in the arithmetic secret share for multiplication operations, and we use ```BeaverProvider``` to simulate a trusted third party to provide auxiliary operation data.

## Party
First, we need to define the parties involved in the computation. For secure two-party computation, we need two parties: the server and the client.
When setting up the parties, we need to specify the address and port for each party. Each party has a tcp server and a tcp client. They all need an address and a port. We also need to set the Beaver triple provider and the wrap provider for the computations. If you are planning to do comparison operations, do not forget to set the compare key provider.
In this demonstration we are using multi-threading to simulate two parties. In real applications, the server and client run in two files. You can refer to ``./ debug/crypto/primitives/arithmetic_secret_sharing/test_ass_server.py`` and ```./ debug/crypto/primitives/arithmetic_secret_sharing/test_ass_client.py```.

In [2]:
import threading

# set Server
server = SemiHonestCS(type='server')

server.set_multiplication_provider()
server.set_comparison_provider()


def set_server():
    # CS connect (self tcp server address) (self tcp client address) (other tcp server address) (other tcp client address)
    server.connect(('127.0.0.1', 8089), ('127.0.0.1', 8088), ('127.0.0.1', 20000), ('127.0.0.1', 20001))



# set Client
client = SemiHonestCS(type='client')

client.set_multiplication_provider()
client.set_comparison_provider()

def set_client():
    # CS connect
   client.connect(('127.0.0.1', 20000), ('127.0.0.1', 20001), ('127.0.0.1', 8089), ('127.0.0.1', 8088))


server_thread = threading.Thread(target=set_server)
client_thread = threading.Thread(target=set_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()

TCPServer waiting for connection ......
TCPServer waiting for connection ......
successfully connect to server 127.0.0.1:8089
TCPServer successfully connected by :('127.0.0.1', 20001)
successfully connect to server 127.0.0.1:20000TCPServer successfully connected by :('127.0.0.1', 8088)


If you see two instances of "successfully connected", it indicates that the communication between the two parties has been established successfully.

## Secret Sharing
If both parties have data that they want to compute on without revealing their individual data to each other, you can use the ```share``` method from ```ArithmeticSecretSharing``` (ASS) to perform data sharing. Additionally, you need to utilize TCP to send each party's shares to the other party and receive their own shares.
In this case, let's assume that the server has data denoted as x, and the client has data denoted as y.

In [3]:
from config.base_configs import DEVICE

# data belong to server
x = RingTensor.convert_to_ring(torch.tensor([[1.0, 2.0], [3.0, 4.0]], device=DEVICE))
# data belong to client
y = RingTensor.convert_to_ring(torch.tensor([[-1.0, 2.0], [4.0, 3.0]], device=DEVICE))

# split x into 2 parts
X = ArithmeticSharedRingTensor.share(x, 2)

# split y into 2 parts
Y = ArithmeticSharedRingTensor.share(y, 2)

# server shares x1 to client
server.send(X[1])
# client receives x1 from server
x1 = client.receive()

# client shares y0 to server
client.send(Y[0])
# server receives y0 from client
y0 = server.receive()

# convert RingTensor to ASS
# server
shared_x_0 = ArithmeticSharedRingTensor(X[0], server)
shared_y_0 = ArithmeticSharedRingTensor(y0, server)

print("\n shared x in server:", shared_x_0)
print("\n shared y in server:", shared_y_0)

# client
shared_x_1 = ArithmeticSharedRingTensor(x1, client)
shared_y_1 = ArithmeticSharedRingTensor(Y[1], client)
print("\n shared x in client:", shared_x_1)
print("\n shared y in client:", shared_y_1)


 shared x in server: ArithmeticSharedRingTensor
 value:tensor([[ 5466333745128403300, -6395018159207208530],
        [  889215137156102987, -4582170111517986136]], device='cuda:0') 
 dtype:float 
 scale:65536
 party:0

 shared y in server: ArithmeticSharedRingTensor
 value:tensor([[ 7272982348057240323, -5783740582566321592],
        [  150646293732089090, -1493090933781434579]], device='cuda:0') 
 dtype:float 
 scale:65536
 party:0

 shared x in client: ArithmeticSharedRingTensor
 value:tensor([[-5466333745128337764,  6395018159207339602],
        [ -889215137155906379,  4582170111518248280]], device='cuda:0') 
 dtype:float 
 scale:65536
 party:1

 shared y in client: ArithmeticSharedRingTensor
 value:tensor([[-7272982348057305859,  5783740582566452664],
        [ -150646293731826946,  1493090933781631187]], device='cuda:0') 
 dtype:float 
 scale:65536
 party:1


## Secret Restoring
If you want to restore the original value by the share, you can use the ```restore()``` method, which returns a ```RingTensor``` value, and then the ```convert_to_real_field``` can recover the result.
In this tutorial, we only print the recovered results on the server side.

In [4]:
# restore share_x
# server
def restore_server():
    restored_x = shared_x_0.restore()
    real_x = restored_x.convert_to_real_field()
    print("\n x after restoring:", real_x)

# client
def restore_client():
    shared_x_1.restore()

server_thread = threading.Thread(target=restore_server)
client_thread = threading.Thread(target=restore_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


 x after restoring: tensor([[1., 2.],
        [3., 4.]], device='cuda:0')


## Operations
Next, we'll show you how to use arithmetic secret sharing to achieve secure two-party computation.

#### Arithmetic Operations

In [5]:
# Addition
# restore result
def addition_server():
    res_0 = shared_x_0 + shared_y_0
    result_restored = res_0.restore().convert_to_real_field()
    print("\nAddition", result_restored)

def addition_client():
    res_1 = shared_x_1 + shared_y_1
    res_1.restore()

server_thread = threading.Thread(target=addition_server)
client_thread = threading.Thread(target=addition_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


Addition tensor([[0., 4.],
        [7., 7.]], device='cuda:0')


In [6]:
# Subtraction
# restore result
def subtraction_server():
    res_0 = shared_x_0 - shared_y_0
    result_restored = res_0.restore().convert_to_real_field()
    print("\nSubtraction", result_restored)

def subtraction_client():
    res_1 = shared_x_1 - shared_y_1
    res_1.restore()

server_thread = threading.Thread(target=subtraction_server)
client_thread = threading.Thread(target=subtraction_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


Subtraction tensor([[ 2.,  0.],
        [-1.,  1.]], device='cuda:0')


In [7]:
# Multiplication
# restore result
def multiplication_server():
    res_0 = shared_x_0 * shared_y_0
    result_restored = res_0.restore().convert_to_real_field()
    print("\nMultiplication", result_restored)

def multiplication_client():
    res_1 = shared_x_1 * shared_y_1
    res_1.restore()

server_thread = threading.Thread(target=multiplication_server)
client_thread = threading.Thread(target=multiplication_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


Multiplication tensor([[-1.,  4.],
        [12., 12.]], device='cuda:0')


Note: Since all the beaver triples used were generated during the offline phase, don't forget to generate the required matrix beaver triples before performing matrix multiplication.

In [8]:
# Matrix Multiplication
def matrix_multiplication_server():
    # gen beaver triples in advance
    triples = MatrixTriples.gen(1, x.shape, y.shape)
    server.providers[MatrixTriples].param = [triples[0]]
    server.send(triples[1])
    server.providers[MatrixTriples].load_mat_beaver()
    res_0 = shared_x_0 @ shared_y_0
    result_restored = res_0.restore().convert_to_real_field()
    print("\nMatrix Multiplication", result_restored)

def matrix_multiplication_client():
    client.providers[MatrixTriples].param = [client.receive()]
    client.providers[MatrixTriples].load_mat_beaver()
    res_1 = shared_x_1 @ shared_y_1
    res_1.restore()

server_thread = threading.Thread(target=matrix_multiplication_server)
client_thread = threading.Thread(target=matrix_multiplication_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


Matrix Multiplication tensor([[ 7.0000,  8.0000],
        [13.0000, 18.0000]], device='cuda:0')


#### Comparison Operations
The output results ```0``` and ```1``` correspond to the ``False`` and ``True`` values obtained from comparing the sizes of the torch tensors.

In [9]:
# Less than
def less_than_server():
    res_0 = shared_x_0 < shared_y_0
    result_restored = res_0.restore().convert_to_real_field()
    print("\n(x < y)", result_restored)

def less_than_client():
    res_1 = shared_x_1 < shared_y_1
    res_1.restore()

server_thread = threading.Thread(target=less_than_server)
client_thread = threading.Thread(target=less_than_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


(x < y) tensor([[0., 0.],
        [1., 0.]], device='cuda:0')


In [10]:
# Less than or equal
def less_equal_server():
    res_0 = shared_x_0 <= shared_y_0
    result_restored = res_0.restore().convert_to_real_field()
    print("\n(x <= y)", result_restored)

def less_equal_client():
    res_1 = shared_x_1 <= shared_y_1
    res_1.restore()

server_thread = threading.Thread(target=less_equal_server)
client_thread = threading.Thread(target=less_equal_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


(x <= y) tensor([[0., 1.],
        [1., 0.]], device='cuda:0')


In [11]:
# Greater than
def greater_than_server():
    res_0 = shared_x_0 > shared_y_0
    result_restored = res_0.restore().convert_to_real_field()
    print("\n(x > y)", result_restored)

def greater_than_client():
    res_1 = shared_x_1 > shared_y_1
    res_1.restore()

server_thread = threading.Thread(target=greater_than_server)
client_thread = threading.Thread(target=greater_than_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


(x > y) tensor([[1., 0.],
        [0., 1.]], device='cuda:0')


In [12]:
# Greater than or equal
def greater_equal_server():
    res_0 = shared_x_0 >= shared_y_0
    result_restored = res_0.restore().convert_to_real_field()
    print("\n(x >= y)", result_restored)

def greater_equal_client():
    res_1 = shared_x_1 >= shared_y_1
    res_1.restore()

server_thread = threading.Thread(target=greater_equal_server)
client_thread = threading.Thread(target=greater_equal_client)

server_thread.start()
client_thread.start()
client_thread.join()
server_thread.join()


(x >= y) tensor([[1., 1.],
        [0., 1.]], device='cuda:0')
